In [1]:
import json
import pandas as pd 
import numpy as np 
from glob import glob
import os
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
folder_path=glob("../../data/*")
data1_img_path=glob(folder_path[0]+"/data/*.jpg")
test_data_img_path=glob(folder_path[1]+"/data/*.jpg")
data2_img_path=glob(folder_path[2]+"/data/*.jpg")
data4_img_path=glob(folder_path[3]+"/data/*.jpg")

def AnnotationsPd(annotations):
    values_list = []
    names_list = [] 
    df = pd.DataFrame(annotations['tags'])
    for row in df['attributes']:
        values_list.append(row[0]['value'])
        names_list.append(row[0]['name'])

    df=df.drop(['attributes'],axis=1)   
    df.insert(3,'value',values_list)
    df.insert(4,'name',names_list)
    return df



In [3]:
data1_img_path.extend(data2_img_path)
data1_img_path.extend(data4_img_path)

In [4]:
Train_file_path=[]
Test_file_path=[]
with open(folder_path[0]+'/annotations.json') as f:
    annotations_json = json.load(f)
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data1_img_path if str(frame).zfill(5) in s] 
    Train_file_path.append(file[0])
with open(folder_path[2]+'/annotations.json') as f:
    annotations_json = json.load(f)   
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data2_img_path if str(frame).zfill(5) in s] 
    Train_file_path.append(file[0])

with open(folder_path[1]+'/annotations.json') as f:
    annotations_json = json.load(f)    
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in test_data_img_path if str(frame).zfill(5) in s] 
    Test_file_path.append(file[0])
    
with open(folder_path[3]+'/annotations.json') as f:
    annotations_json = json.load(f)    
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data4_img_path if str(frame).zfill(5) in s] 
    Train_file_path.append(file[0])
Train_file_path=list(set(Train_file_path))
Test_file_path=list(set(Test_file_path))

In [5]:
Train_standard_file=[]
Train_TF_standard=[]
Test_standard_file=[]
Test_TF_standard=[]
for i in range(len(Train_file_path)):
    Train_TF_standard.append(1)
    shutil.copy(Train_file_path[i],'../../data/original_dataset/StandardFrame/Train/'+str(i)+'.jpg')
    Train_standard_file.append('StandardFrame/Train/'+str(i)+'.jpg')   
for i in range(len(Test_file_path)):
    Test_TF_standard.append(1)
    shutil.copy(Test_file_path[i],'../../data/original_dataset/StandardFrame/Test/'+str(i)+'.jpg')
    Test_standard_file.append('StandardFrame/Test/'+str(i)+'.jpg') 

In [6]:
Train_not_standard_file1=list(set(data1_img_path)-set(Train_file_path))
Test_not_standard_file1=list(set(test_data_img_path)-set(Test_file_path))

for i in range(len(Train_not_standard_file1)):
    Train_TF_standard.append(0)
    shutil.copy(Train_not_standard_file1[i],'../../data/original_dataset/NotStandardFrame/Train/'+str(i)+'.jpg')
    Train_standard_file.append('NotStandardFrame/Train/'+str(i)+'.jpg')
for i in range(len(Test_not_standard_file1)):
    Test_TF_standard.append(0)
    shutil.copy(Test_not_standard_file1[i],'../../data/original_dataset/NotStandardFrame/Test/'+str(i)+'.jpg')
    Test_standard_file.append('NotStandardFrame/Test/'+str(i)+'.jpg')

In [9]:
Train_original_standard_frame = pd.DataFrame({'file_path':Train_standard_file,
                             'standard_frame':Train_TF_standard})
Test_original_standard_frame = pd.DataFrame({'file_path':Test_standard_file,
                             'standard_frame':Test_TF_standard})
Train_original_standard_frame=shuffle(Train_original_standard_frame)
Test_original_standard_frame=shuffle(Test_original_standard_frame)
Test_original_standard_frame


,file_path,standard_frame
1446,NotStandardFrame/Test/957.jpg,0
1393,NotStandardFrame/Test/904.jpg,0
952,NotStandardFrame/Test/463.jpg,0
553,NotStandardFrame/Test/64.jpg,0
427,StandardFrame/Test/427.jpg,1
...,...,...
48,StandardFrame/Test/48.jpg,1
1472,NotStandardFrame/Test/983.jpg,0
1218,NotStandardFrame/Test/729.jpg,0
1897,NotStandardFrame/Test/1408.jpg,0


In [10]:
Train_original_standard_frame.to_csv('../../data/original_dataset/Train_data_set.csv', index = None) #csv파일로 생성
Test_original_standard_frame.to_csv('../../data/original_dataset/Test_data_set.csv', index = None) #csv파일로 생성